<a href="https://colab.research.google.com/github/GabrielDeSixto/Practica-Final-IA-Center/blob/main/Practica_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Practica Final Caso Covid 2019 México**

***Paso 1: Carga de datos***

*  Descargas y descomprimes un conjunto de datos de COVID-19 en formato CSV.

*  Utilizas pandas para cargar los datos en un DataFrame llamado df.

*  Realizas algunas operaciones de preprocesamiento en los datos, como eliminar columnas innecesarias.

In [ ]:
!wget https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

--2023-11-07 04:28:41--  https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 201.98.60.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|201.98.60.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17989229 (17M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip.1’

datos_abiertos_covi 100%[===================>]  17.16M  6.36MB/s    in 2.7s    

2023-11-07 04:28:44 (6.36 MB/s) - ‘datos_abiertos_covid19.zip.1’ saved [17989229/17989229]



In [ ]:
! unzip /content/datos_abiertos_covid19.zip

Archive:  /content/datos_abiertos_covid19.zip
replace COVID19MEXICO.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: COVID19MEXICO.csv       


In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('COVID19MEXICO.csv', low_memory=False)
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183124 entries, 0 to 1183123
Data columns (total 40 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   FECHA_ACTUALIZACION    1183124 non-null  object
 1   ID_REGISTRO            1183124 non-null  object
 2   ORIGEN                 1183124 non-null  int64 
 3   SECTOR                 1183124 non-null  int64 
 4   ENTIDAD_UM             1183124 non-null  int64 
 5   SEXO                   1183124 non-null  int64 
 6   ENTIDAD_NAC            1183124 non-null  int64 
 7   ENTIDAD_RES            1183124 non-null  int64 
 8   MUNICIPIO_RES          1183124 non-null  int64 
 9   TIPO_PACIENTE          1183124 non-null  int64 
 10  FECHA_INGRESO          1183124 non-null  object
 11  FECHA_SINTOMAS         1183124 non-null  object
 12  FECHA_DEF              1183124 non-null  object
 13  INTUBADO               1183124 non-null  int64 
 14  NEUMONIA               1183124 non

In [ ]:
df=data

In [ ]:
df = df.drop(columns=['SECTOR','MUNICIPIO_RES','ID_REGISTRO',
                      'ORIGEN','PAIS_NACIONALIDAD','PAIS_ORIGEN',
                      'INDIGENA','ENTIDAD_UM','ENTIDAD_NAC','NACIONALIDAD',
                      'ENTIDAD_RES'])
df.columns

Index(['FECHA_ACTUALIZACION', 'SEXO', 'TIPO_PACIENTE', 'FECHA_INGRESO',
       'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA', 'EDAD',
       'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES', 'EPOC', 'ASMA',
       'INMUSUPR', 'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB',
       'RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO',
       'CLASIFICACION_FINAL', 'MIGRANTE', 'UCI'],
      dtype='object')

***Paso 2: Preprocesamiento de datos***

Codificamos las variables categóricas "SEXO" y "TIPO_PACIENTE" utilizando
LabelEncoder.
Calculas características basadas en las fechas, como "DIAS_DEFUNCION", "DIAS_HOSPITALIZADO" y "DIAS_SINTOMAS".

Realizas un ajuste en "DIAS_DEFUNCION" para asegurarte de que los valores negativos se establezcan en cero.

Tambien ponemos los valores vacios o que sean menores a 0 como NaN

In [ ]:
df['FECHA_ACTUALIZACION']=pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_INGRESO']=pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS']=pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_DEF']=pd.to_datetime(df['FECHA_DEF'], errors='coerce')
df

,FECHA_ACTUALIZACION,SEXO,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,EMBARAZO,...,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,UCI
0,2023-10-31,2,1,2023-03-14,2023-03-10,NaT,97,2,47,97,...,2,2,2,2,97,1,2,7,99,97
1,2023-10-31,2,1,2023-01-01,2023-01-01,NaT,97,2,39,97,...,2,2,2,2,97,2,97,6,99,97
2,2023-10-31,1,2,2023-01-01,2023-01-01,NaT,2,2,2,2,...,2,2,2,2,97,2,97,6,99,2
3,2023-10-31,2,1,2023-03-04,2023-03-01,NaT,97,2,14,97,...,2,2,2,2,97,1,2,7,99,97
4,2023-10-31,2,1,2023-04-15,2023-04-15,NaT,97,2,51,97,...,2,2,2,2,97,2,97,6,99,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183119,2023-10-31,2,1,2023-08-30,2023-08-30,NaT,97,99,61,97,...,2,2,99,2,97,1,2,7,99,97
1183120,2023-10-31,1,1,2023-08-30,2023-08-30,NaT,97,99,61,2,...,2,2,99,2,97,1,2,7,99,97
1183121,2023-10-31,1,1,2023-08-30,2023-08-30,NaT,97,99,39,2,...,2,2,99,2,97,1,2,7,99,97
1183122,2023-10-31,2,1,2023-08-31,2023-08-31,NaT,97,99,66,97,...,2,2,99,2,97,1,2,7,99,97


In [ ]:
for col in ['TIPO_PACIENTE']:
  df[col]=df[col].astype(str)


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in ['TIPO_PACIENTE', 'SEXO']:
  df[col]=le.fit_transform(df[col])

#crear caracteristicas a partir de las fechas

df['DIAS_DEFUNCION']=(df['FECHA_ACTUALIZACION']-df['FECHA_DEF']).dt.days
df['DIAS_HOSPITALIZADO']=(df['FECHA_ACTUALIZACION']-df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS']=(df['FECHA_ACTUALIZACION']-df['FECHA_SINTOMAS']).dt.days



#read neuronal revisar esto
df['DIAS_DEFUNCION']=df['DIAS_DEFUNCION'].apply(lambda x:0 if x  < 0 else x)
df['DIAS_DEFUNCION']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
1183119   NaN
1183120   NaN
1183121   NaN
1183122   NaN
1183123   NaN
Name: DIAS_DEFUNCION, Length: 1183124, dtype: float64

In [ ]:
df = df.drop(columns=['FECHA_ACTUALIZACION', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'DIAS_DEFUNCION'])

In [ ]:
X=df.drop(columns=['CLASIFICACION_FINAL'])
y=df['CLASIFICACION_FINAL']

***Paso 3: División de datos***

Divides los datos en conjuntos de entrenamiento (X_train, y_train) y prueba (X_test, y_test) para entrenar y evaluar el modelo.

In [ ]:
from sklearn.model_selection import train_test_split
# Dividir el dataset en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train=X_train.astype('float32')
y_train=y_train.astype('float32')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183124 entries, 0 to 1183123
Data columns (total 27 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   SEXO                   1183124 non-null  int64
 1   TIPO_PACIENTE          1183124 non-null  int64
 2   INTUBADO               1183124 non-null  int64
 3   NEUMONIA               1183124 non-null  int64
 4   EDAD                   1183124 non-null  int64
 5   EMBARAZO               1183124 non-null  int64
 6   HABLA_LENGUA_INDIG     1183124 non-null  int64
 7   DIABETES               1183124 non-null  int64
 8   EPOC                   1183124 non-null  int64
 9   ASMA                   1183124 non-null  int64
 10  INMUSUPR               1183124 non-null  int64
 11  HIPERTENSION           1183124 non-null  int64
 12  OTRA_COM               1183124 non-null  int64
 13  CARDIOVASCULAR         1183124 non-null  int64
 14  OBESIDAD               1183124 non-null  int64
 15

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clf=RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.21      0.05      0.07      1670
           2       1.00      0.40      0.57         5
           3       1.00      1.00      1.00     83882
           4       0.09      0.03      0.05        29
           5       0.51      0.50      0.50       391
           6       0.95      0.96      0.95     12457
           7       0.99      1.00      0.99    138191

    accuracy                           0.99    236625
   macro avg       0.68      0.56      0.59    236625
weighted avg       0.98      0.99      0.99    236625



In [ ]:
import cuml
from cuml.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf=RandomForestClassifier(random_state=42)



clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.89      0.01      0.03      1670
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     83882
           4       0.00      0.00      0.00        29
           5       0.56      0.28      0.38       391
           6       0.94      0.97      0.96     12457
           7       0.99      1.00      0.99    138191

    accuracy                           0.99    236625
   macro avg       0.63      0.47      0.48    236625
weighted avg       0.99      0.99      0.99    236625



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***Paso 4: Entrenamiento y evaluación del modelo***

*   Utilizas la biblioteca cuML (RAPIDS) para crear un clasificador de Bosques

*   Aleatorios y un clasificador de k-Vecinos más Cercanos (KNN).

*   Iteras a través de los clasificadores y entrenas cada uno con los datos de entrenamiento.

*   Luego, realizas predicciones en los datos de prueba y generas informes de clasificación para evaluar el rendimiento de los modelos RandomForestClassifier y KNeighborsClassifier

In [ ]:
import cuml
from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

cuml_RFC = RandomForestClassifier(random_state=42)
cuml_KNF = KNeighborsClassifier()

classifiers = [
    ('Random Forest', cuml_RFC),
    ('KNN', cuml_KNF),
]

for name, clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(f"Classification Report for {name}:")
    print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


Classification Report for Random Forest:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.89      0.01      0.03      1670
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00     83882
           4       0.00      0.00      0.00        29
           5       0.56      0.28      0.38       391
           6       0.94      0.97      0.96     12457
           7       0.99      1.00      0.99    138191

    accuracy                           0.99    236625
   macro avg       0.63      0.47      0.48    236625
weighted avg       0.99      0.99      0.99    236625

Classification Report for KNN:
              precision    recall  f1-score   support

           1       0.34      0.02      0.03      1670
           2       0.67      0.40      0.50         5
           3       0.94      0.83      0.88     83882
           4       0.00      0.00      0.00        29
           5       0.22      0.03      0.05       391
           6       0.96      0.92      0.94    

***Paso 5: Interfaz de usuario con Gradio***

Creamos una interfaz de usuario interactiva utilizando Gradio para ingresar los valores de entrada necesarios para la predicción.

Los valores de entrada incluyen información sobre el paciente, como sexo, tipo de paciente, edad y comorbilidades.

La función predict toma los valores de entrada, crea un DataFrame con ellos y utiliza el modelo entrenado para hacer una predicción.
La predicción se muestra en un cuadro de texto en la interfaz de usuario.

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install gradio

***Paso 6: Ejecución de la aplicación web***

Ejecutamos la aplicación web utilizando demo.launch() para permitir a los usuarios ingresar datos y obtener predicciones.

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np


def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
             diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
           uci, dias_hospitalizacion, dias_sintomas):


    # Crea un DataFrame con los valores de entrada
    input_data = pd.DataFrame([[
        sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
       diabetes, epoc, asma, inmusupr,
        hipertension, otra_com, cardiovascular, obesidad,
        renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
        resultado_lab, toma_muestra_antigeno, resultado_antigeno, uci, dias_hospitalizacion, dias_sintomas
        ]], columns=[
        'SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO','DIABETES', 'EPOC',
        'ASMA', 'INMUSUPR','HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD','RENAL_CRONICA',
        'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB','RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO',
        'RESULTADO_ANTIGENO','UCI', 'DIAS_HOSPITALIZACION', 'DIAS_SINTOMAS'])

    prediction = clf.predict(input_data)
    return prediction




gr.Row
inputs = [
    gr.Radio(choices=[0, 1], label='Sexo'),
    gr.Radio(choices=[0, 1], label='Tipo de Paciente'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Intubado'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Neumonía'),
    gr.Slider(minimum=0, maximum=120, value=30, label='Edad'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Embarazo'),
    gr.Radio(choices=[1, 2, 98], label='Diabetes'),
    gr.Radio(choices=[1, 2, 98], label='EPOC'),
    gr.Radio(choices=[1, 2, 98], label='Asma'),
    gr.Radio(choices=[1, 2, 98], label='Inmunosupresión'),
    gr.Radio(choices=[1, 2, 98], label='Hipertensión'),
    gr.Radio(choices=[1, 2, 98], label='Otra Comorbilidad'),
    gr.Radio(choices=[1, 2, 98], label='Cardiovascular'),
    gr.Radio(choices=[1, 2, 98], label='Obesidad'),
    gr.Radio(choices=[1, 2, 98], label='Enfermedad Renal Crónica'),
    gr.Radio(choices=[1, 2, 98], label='Tabaquismo'),
    gr.Radio(choices=[1, 2, 99], label='Contacto con otro caso'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de laboratorio'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de laboratorio'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='UCI'),
    gr.Number(label='Días de Hospitalización',value=0),
    gr.Number(label='Días con Síntomas',value=0),
]

# Crear componente de salida para Gradio
outputs = gr.Textbox(label="Predicción")

# Crear la interfaz de Gradio
demo = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)

# Ejecutar la aplicación web
if __name__ == "__main__":
    demo.launch(show_api=False,debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://dfc81a8cf814261344.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 427, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1484, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1106, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    re

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dfc81a8cf814261344.gradio.live
